# Trends in Caribbean atmospheric stability

Plot trends in convective available potential energy (CAPE), convective inhibition (CIN) and 300-hPa temperatures over the Caribbean and tropical North Atlantic regions.

In [ ]:
# Package imports
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
from cartopy.mpl.ticker import LatitudeFormatter, LongitudeFormatter
import cmaps as ncl_cm
import pymannkendall as mk

from cartopy.mpl.geoaxes import GeoAxes
from mpl_toolkits.axes_grid1 import AxesGrid
import matplotlib.ticker as mticker


In [ ]:
# Figure format defaults
plt.rcParams['figure.figsize'] = 12, 6
plt.rcParams['figure.titlesize'] = 15
plt.rcParams["axes.titlesize"] = 15

# Define AxesGrid parameters to set map projections and orientation
projection = ccrs.PlateCarree(central_longitude=-180)
mproj = ccrs.PlateCarree()
axes_class = (GeoAxes, dict(projection=projection))

## Retrieve data
Trend data is generated using the ECMWF fifth generation reanalysis (ERA5) with 2.5°x2.5° horizontal grid resolution.

In [ ]:
# ERA5 data
# SST fields have been linearly interpolated (cdo) to remove nans
ds_sst = xr.open_dataset('../data/era5_monthly_sst_lowres_1940-2024_interp.nc').rename({'valid_time': 'time'})-273.15
ds_prcp = xr.open_dataset('../data/era5_monthly_conv_precip_rate_0.25x0.25_1940-2024.nc').rename({'valid_time': 'time'})*86400 # convert mm/s to mm/day
ds_temp = xr.open_dataset('../data/era5_monthly_temp300_cres_1940-2024.nc').rename({'valid_time':'time'})-273.15
ds_cape = xr.open_dataset('../data/era5_monthly_cape_lowres_1940-2024.nc').rename({'valid_time':'time'})
ds_cin = xr.open_dataset('../data/era5_monthly_cin_lowres_1940-2024.nc').rename({'valid_time':'time'})


## Trend calculations

### Calculate the trend at each grid point: SST, T300

In [ ]:
# Calculate the spatial linear trends for T300 across the tropical North Atlantic
''' Calculate the linear trend on every xarray DataArray chunk. '''
def calculate_spatial_trend(chunk):
    # Create an empty DataArray to store the trend values
    trend = xr.zeros_like(chunk.isel(year=0))
    p_value = xr.zeros_like(chunk.isel(year=0))

    # Loop through each grid point
    for lat in chunk.latitude:
        for lon in chunk.longitude:
            # Extract time series for the current grid point
            y = chunk.sel(latitude=lat, longitude=lon).values

            # Check if the time series has enough data points for trend calculation
            # Modified condition to require at least 2 valid data points
            if len(y) > 20 and not np.all(np.isnan(y)) and np.sum(~np.isnan(y)) >= 20 and np.divide(np.sum(np.isnan(y)),len(y))<0.1:
                # Run the Mann-Kendall trend calculation and hypothesis test
                mk_result = mk.original_test(y)

                # Store the trend (slope) and p-value
                trend.loc[dict(latitude=lat, longitude=lon)] = mk_result.slope
                p_value.loc[dict(latitude=lat, longitude=lon)] = mk_result.p  # Extract p-value for the slope
            else:
                # Handle cases with insufficient data (e.g., set trend and p-value to NaN)
                trend.loc[dict(latitude=lat, longitude=lon)] = 0
                p_value.loc[dict(latitude=lat, longitude=lon)] = np.nan

    ds = xr.Dataset({'trend': trend, 'p_value': p_value})
    return ds # Return a Dataset instead of a tuple

def apply_func(da):
    seasonal_da = da.sel(time=da.time.dt.month.isin(np.arange(5, 8, 1))).groupby('time.year').mean('time')
    seasonal_da = seasonal_da.sel(year=slice(1979, 2024))
    dask_da = seasonal_da.chunk({'latitude': 10, 'longitude': 10})

    # Create template array
    array1 = dask_da.isel(year=0)
    array2 = dask_da.isel(year=0)
    template = xr.Dataset({'trend': array1, 'p_value': array2})

    trend_results = dask_da.map_blocks(calculate_spatial_trend, template=template).compute()

    return trend_results

#---- Apply function ----
temp_trend_results = apply_func(ds_temp.t.sel(pressure_level=300))
sst_trend_results = apply_func(ds_sst.sst)

### Calculate the trend at each grid point: CAPE, CIN

In [ ]:
# Calculate the spatial linear trends for CAPE, CIN across the tropical North Atlantic
''' Calculate the linear trend on every xarray DataArray chunk. '''
def calculate_spatial_trend(chunk):
    # Create an empty DataArray to store the trend values
    trend = xr.zeros_like(chunk.isel(year=0))
    p_value = xr.zeros_like(chunk.isel(year=0))

    # Loop through each grid point
    for lat in chunk.latitude:
        for lon in chunk.longitude:
            # Extract time series for the current grid point
            y = chunk.sel(latitude=lat, longitude=lon).values

            # Check if the time series has enough data points for trend calculation
            # Modified condition to require at least 2 valid data points
            if len(y) > 20 and not np.all(np.isnan(y)) and np.sum(~np.isnan(y)) >= 20 and np.divide(np.sum(np.isnan(y)),len(y))<0.1:
                # Run the Mann-Kendall trend calculation and hypothesis test
                mk_result = mk.original_test(y)

                # Store the trend (slope) and p-value
                trend.loc[dict(latitude=lat, longitude=lon)] = mk_result.slope
                p_value.loc[dict(latitude=lat, longitude=lon)] = mk_result.p  # Extract p-value for the slope
            else:
                # Handle cases with insufficient data (e.g., set trend and p-value to NaN)
                trend.loc[dict(latitude=lat, longitude=lon)] = 0
                p_value.loc[dict(latitude=lat, longitude=lon)] = np.nan

    ds = xr.Dataset({'trend': trend, 'p_value': p_value})
    return ds # Return a Dataset instead of a tuple

def apply_func(da):
    seasonal_da = da.sel(time=da.time.dt.month.isin(np.arange(5, 8, 1))).groupby('time.year').mean('time')
    seasonal_da = seasonal_da.sel(year=slice(1979, 2024))
    dask_da = seasonal_da.chunk({'latitude': 10, 'longitude': 10})

    # Create template array
    array1 = dask_da.isel(year=0)
    array2 = dask_da.isel(year=0)
    template = xr.Dataset({'trend': array1, 'p_value': array2})

    trend_results = dask_da.map_blocks(calculate_spatial_trend, template=template).compute()

    return trend_results

#---- Apply function ----
cape_trend_results = apply_func(ds_cape.cape)
cin_trend_results = apply_func(ds_cin.cin)
